# Check the completeness of 2019 Semiannul geomeidian data

In [1]:
import pandas as pd
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import re

In [2]:
s3 = boto3.resource('s3', config=Config(signature_version=UNSIGNED))

In [3]:
bkt = s3.Bucket('deafrica-data-dev-af')

In [4]:
objs = [o for o in bkt.objects.filter(Prefix='gm_s2_semiannual/0-1-0/')]

In [5]:
len(objs)

139292

## Failed task according to grafana
```
	2021-04-09 01:53:19	ip-10-85-41-136.af-south-1.compute.internalstats-gm-semiannual-v49kr	
{"log":"[2021-04-08 15:53:19,178] {_cli_run.py:157} INFO - Skipped task #74: x207/y103/2019-07--P6M eabcdd59-c236-5860-b608-767181235798
","stream":"stdout"}
	
2021-04-09 01:53:19	ip-10-85-41-136.af-south-1.compute.internalstats-gm-semiannual-v49kr	
{"log":"[2021-04-08 15:53:19,172] {proc.py:185} INFO - Skipped task @ s3://deafrica-data-dev-af/gm_s2_semiannual/0-1-0/x207/y103/2019-07--P6M/gm_s2_semiannual_x207y103_2019-07--P6M.json
","stream":"stdout"}
	
2021-04-09 01:53:19	ip-10-85-41-136.af-south-1.compute.internalstats-gm-semiannual-v49kr	
{"log":"[2021-04-08 15:53:19,144] {proc.py:174} INFO - Starting processing of x207/y103/2019-07--P6M
","stream":"stdout"}
	
2021-04-09 01:53:04	ip-10-85-81-129.af-south-1.compute.internalstats-gm-semiannual-pgxgq	
{"log":"[2021-04-08 15:53:04,292] {_cli_run.py:163} ERROR - Failed task #69: x207/y103/2019-07--P6M eabcdd59-c236-5860-b608-767181235798
","stream":"stdout"}
	
2021-04-09 01:53:04	ip-10-85-81-129.af-south-1.compute.internalstats-gm-semiannual-pgxgq	
{"log":"[2021-04-08 15:53:04,235] {proc.py:162} ERROR - Error during processing of x207/y103/2019-07--P6M Timed out trying to connect to 'inproc://10.85.86.166/1/1' after 10 s: Timed out trying to connect to 'inproc://10.85.86.166/1/1' after 10 s: connect() didn't finish in time
","stream":"stdout"}
	
2021-04-09 01:45:53	ip-10-85-81-129.af-south-1.compute.internalstats-gm-semiannual-pgxgq	
{"log":"[2021-04-08 15:45:53,761] {proc.py:174} INFO - Starting processing of x207/y103/2019-07--P6M
","stream":"stdout"}Show context
```

In [6]:
df = pd.read_csv('../testing/eastern_cropmask/data/s2_l2a_all.csv')

In [7]:
df2019 = df[df['T'].apply(lambda T: T.startswith('2019'))]

In [8]:
df2019.head()

,T,X,Y,datasets,days
17599,2019-01--P6M,153,98,36,36
17600,2019-01--P6M,153,99,74,36
17601,2019-01--P6M,153,100,36,36
17602,2019-01--P6M,154,95,4,2
17603,2019-01--P6M,154,96,52,36


In [9]:
len(df2019)

8727

In [10]:
m = re.compile(r'x\d{3}/y\d{3}')

In [11]:
jsons = [o for o in objs if o.key.endswith('json') and m.search(o.key)]

In [12]:
len(jsons)

8705

In [14]:
len(df2019) - len(jsons)

22

In [35]:
jsons[:15]

[s3.ObjectSummary(bucket_name='deafrica-data-dev-af', key='gm_s2_semiannual/0-1-0/x153/y098/2019-01--P6M/gm_s2_semiannual_x153y098_2019-01--P6M.json'),
 s3.ObjectSummary(bucket_name='deafrica-data-dev-af', key='gm_s2_semiannual/0-1-0/x153/y098/2019-07--P6M/gm_s2_semiannual_x153y098_2019-07--P6M.json'),
 s3.ObjectSummary(bucket_name='deafrica-data-dev-af', key='gm_s2_semiannual/0-1-0/x153/y099/2019-01--P6M/gm_s2_semiannual_x153y099_2019-01--P6M.json'),
 s3.ObjectSummary(bucket_name='deafrica-data-dev-af', key='gm_s2_semiannual/0-1-0/x153/y099/2019-07--P6M/gm_s2_semiannual_x153y099_2019-07--P6M.json'),
 s3.ObjectSummary(bucket_name='deafrica-data-dev-af', key='gm_s2_semiannual/0-1-0/x153/y100/2019-01--P6M/gm_s2_semiannual_x153y100_2019-01--P6M.json'),
 s3.ObjectSummary(bucket_name='deafrica-data-dev-af', key='gm_s2_semiannual/0-1-0/x153/y100/2019-07--P6M/gm_s2_semiannual_x153y100_2019-07--P6M.json'),
 s3.ObjectSummary(bucket_name='deafrica-data-dev-af', key='gm_s2_semiannual/0-1-0/x154/y

In [36]:
from collections import defaultdict

In [43]:
key_m = re.compile(r'x\d{3}/y\d{3}/2019-\d{2}--P6M')
check_dict = defaultdict(list)

for o in objs:
    groups = key_m.search(o.key)
    if groups:
        check_dict[groups[0]].append(o)
        

In [44]:
groups[0]

'x246/y051/2019-07--P6M'

In [45]:
len(check_dict)

8705

In [46]:
check_dict[groups[0]]

[s3.ObjectSummary(bucket_name='deafrica-data-dev-af', key='gm_s2_semiannual/0-1-0/x246/y051/2019-07--P6M/gm_s2_semiannual_x246y051_2019-07--P6M.json'),
 s3.ObjectSummary(bucket_name='deafrica-data-dev-af', key='gm_s2_semiannual/0-1-0/x246/y051/2019-07--P6M/gm_s2_semiannual_x246y051_2019-07--P6M_B02.tif'),
 s3.ObjectSummary(bucket_name='deafrica-data-dev-af', key='gm_s2_semiannual/0-1-0/x246/y051/2019-07--P6M/gm_s2_semiannual_x246y051_2019-07--P6M_B03.tif'),
 s3.ObjectSummary(bucket_name='deafrica-data-dev-af', key='gm_s2_semiannual/0-1-0/x246/y051/2019-07--P6M/gm_s2_semiannual_x246y051_2019-07--P6M_B04.tif'),
 s3.ObjectSummary(bucket_name='deafrica-data-dev-af', key='gm_s2_semiannual/0-1-0/x246/y051/2019-07--P6M/gm_s2_semiannual_x246y051_2019-07--P6M_B05.tif'),
 s3.ObjectSummary(bucket_name='deafrica-data-dev-af', key='gm_s2_semiannual/0-1-0/x246/y051/2019-07--P6M/gm_s2_semiannual_x246y051_2019-07--P6M_B06.tif'),
 s3.ObjectSummary(bucket_name='deafrica-data-dev-af', key='gm_s2_semiannu

In [47]:
len(check_dict[groups[0]])

16

In [48]:
for key in check_dict:
    assert len(check_dict[key]) == 16

In [51]:
df2019[['X', 'Y', 'T']]

,X,Y,T
17599,153,98,2019-01--P6M
17600,153,99,2019-01--P6M
17601,153,100,2019-01--P6M
17602,154,95,2019-01--P6M
17603,154,96,2019-01--P6M
...,...,...,...
26321,244,51,2019-07--P6M
26322,245,50,2019-07--P6M
26323,245,51,2019-07--P6M
26324,246,50,2019-07--P6M


In [58]:
missing = []

for row in df2019.itertuples():
    fake_key = f'x{row.X:03d}/y{row.Y:03d}/{row.T}'
    if fake_key not in check_dict:
        missing.append((row.Index, fake_key))

In [59]:
row

Pandas(Index=26325, T='2019-07--P6M', X=246, Y=51, datasets=62, days=35)

In [62]:
indices_miss = sorted([row[0] for row in missing])

In [70]:
task_indices = []

start = indices_miss[0]

for a, b in zip(indices_miss[:-1], indices_miss[1:]):
    if b - a > 1:
        task_indices.append((start, a+1))
        start = b
        
if start <= indices_miss[-1]:
    task_indices.append((start, indices_miss[-1]+1))
        

In [71]:
task_indices

[(18184, 18186),
 (18229, 18234),
 (18276, 18280),
 (22542, 22544),
 (22587, 22592),
 (22634, 22638)]

In [69]:
indices_miss

[18184,
 18185,
 18229,
 18230,
 18231,
 18232,
 18233,
 18276,
 18277,
 18278,
 18279,
 22542,
 22543,
 22587,
 22588,
 22589,
 22590,
 22591,
 22634,
 22635,
 22636,
 22637]

In [73]:
missing

[(18184, 'x177/y060/2019-01--P6M'),
 (18185, 'x177/y061/2019-01--P6M'),
 (18229, 'x178/y057/2019-01--P6M'),
 (18230, 'x178/y058/2019-01--P6M'),
 (18231, 'x178/y059/2019-01--P6M'),
 (18232, 'x178/y060/2019-01--P6M'),
 (18233, 'x178/y061/2019-01--P6M'),
 (18276, 'x179/y058/2019-01--P6M'),
 (18277, 'x179/y059/2019-01--P6M'),
 (18278, 'x179/y060/2019-01--P6M'),
 (18279, 'x179/y061/2019-01--P6M'),
 (22542, 'x177/y060/2019-07--P6M'),
 (22543, 'x177/y061/2019-07--P6M'),
 (22587, 'x178/y057/2019-07--P6M'),
 (22588, 'x178/y058/2019-07--P6M'),
 (22589, 'x178/y059/2019-07--P6M'),
 (22590, 'x178/y060/2019-07--P6M'),
 (22591, 'x178/y061/2019-07--P6M'),
 (22634, 'x179/y058/2019-07--P6M'),
 (22635, 'x179/y059/2019-07--P6M'),
 (22636, 'x179/y060/2019-07--P6M'),
 (22637, 'x179/y061/2019-07--P6M')]

In [87]:
lines = """
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:11,062] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/4/S2B_30LXK_20190419_0_L2A/B12.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:11,055] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/6/S2B_30LXK_20190628_0_L2A/B12.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:11,046] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/6/S2B_30LXK_20190608_0_L2A/B12.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:11,037] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/6/S2B_30LXK_20190618_0_L2A/B12.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:11,036] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/4/S2B_30LXK_20190409_0_L2A/B12.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:11,024] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/5/S2B_30LXK_20190509_0_L2A/B12.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:11,014] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/5/S2B_30LXK_20190519_0_L2A/B12.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:11,013] {proc.py:162} ERROR - Error during processing of x179/y058/2019-01--P6M '/vsis3/deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/3/S2B_30LXK_20190330_0_L2A/SCL.tif' not recognized as a supported file format.
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:11,004] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/5/S2B_30LXK_20190529_0_L2A/B12.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,994] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/4/S2B_30LXK_20190429_0_L2A/B12.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,976] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/2/S2B_30LXK_20190208_0_L2A/B12.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,976] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/3/S2B_30LXK_20190330_0_L2A/B12.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,969] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/2/S2B_30LXK_20190228_0_L2A/B12.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,969] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/6/S2B_30LXK_20190608_0_L2A/B11.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,955] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/3/S2B_30LXK_20190320_0_L2A/B12.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,954] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/6/S2B_30LXK_20190618_0_L2A/B11.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,952] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/2/S2B_30LXK_20190218_0_L2A/B12.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,945] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/6/S2B_30LXK_20190628_0_L2A/B11.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,945] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/1/S2B_30LXK_20190119_0_L2A/B12.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,938] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/1/S2B_30LXK_20190109_0_L2A/B12.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,932] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/1/S2B_30LXK_20190129_0_L2A/B12.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,920] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/5/S2B_30LXK_20190529_0_L2A/B11.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,919] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/2/S2B_30LXK_20190228_0_L2A/B11.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,916] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/5/S2B_30LXK_20190519_0_L2A/B11.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,906] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/1/S2B_30LXK_20190119_0_L2A/B11.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,906] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/1/S2B_30LXK_20190129_0_L2A/B11.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,906] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/4/S2B_30LXK_20190429_0_L2A/B11.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,902] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/4/S2B_30LXK_20190409_0_L2A/B11.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,901] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/3/S2B_30LXK_20190320_0_L2A/B11.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,894] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/3/S2B_30LXK_20190330_0_L2A/B11.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,883] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/6/S2B_30LXK_20190618_0_L2A/B8A.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,881] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/4/S2B_30LXK_20190419_0_L2A/B11.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,881] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/4/S2B_30LXK_20190429_0_L2A/B8A.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,881] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/5/S2B_30LXK_20190529_0_L2A/B8A.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,881] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/1/S2B_30LXK_20190109_0_L2A/B11.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,872] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/6/S2B_30LXK_20190628_0_L2A/B8A.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,872] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/3/S2B_30LXK_20190330_0_L2A/B8A.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,871] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/2/S2B_30LXK_20190208_0_L2A/B11.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,871] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/2/S2B_30LXK_20190218_0_L2A/B11.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,871] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/3/S2B_30LXK_20190320_0_L2A/B8A.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,870] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/1/S2B_30LXK_20190129_0_L2A/B8A.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,870] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/5/S2B_30LXK_20190509_0_L2A/B8A.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,869] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/4/S2B_30LXK_20190409_0_L2A/B8A.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,869] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/2/S2B_30LXK_20190228_0_L2A/B8A.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,869] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/6/S2B_30LXK_20190608_0_L2A/B8A.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,868] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/5/S2B_30LXK_20190519_0_L2A/B8A.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,863] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/4/S2B_30LXK_20190419_0_L2A/B8A.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,862] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/2/S2B_30LXK_20190218_0_L2A/B8A.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,855] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/5/S2B_30LXK_20190509_0_L2A/B08.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,854] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/6/S2B_30LXK_20190608_0_L2A/B08.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,850] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/5/S2B_30LXK_20190529_0_L2A/B08.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,845] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/6/S2B_30LXK_20190618_0_L2A/B08.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,837] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/1/S2B_30LXK_20190119_0_L2A/B8A.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,836] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/2/S2B_30LXK_20190228_0_L2A/B08.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,834] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/6/S2B_30LXK_20190628_0_L2A/B08.tif
","stream":"stdout"}
	
2021-04-09 15:40:11	ip-10-85-99-15.af-south-1.compute.internalstats-gm-semiannual-chks9	
{"log":"[2021-04-09 05:40:10,827] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/2/S2B_30LXK_20190208_0_L2A/B8A.tif
","stream":"stdout"}
"""

In [88]:
lines = [l.strip() for l in lines.split('\n') if l.strip() and 'log' in l.strip()]

In [89]:
len(lines)

56

In [90]:
lines

['{"log":"[2021-04-09 05:40:11,062] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/4/S2B_30LXK_20190419_0_L2A/B12.tif',
 '{"log":"[2021-04-09 05:40:11,055] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/6/S2B_30LXK_20190628_0_L2A/B12.tif',
 '{"log":"[2021-04-09 05:40:11,046] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/6/S2B_30LXK_20190608_0_L2A/B12.tif',
 '{"log":"[2021-04-09 05:40:11,037] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/6/S2B_30LXK_20190618_0_L2A/B12.tif',
 '{"log":"[2021-04-09 05:40:11,036] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/4/S2B_30LXK_20190409_0_L2A/B12.tif',
 '{"log":"[2021-04-09 05:40:11,024] {_rio.py:202} ERROR - Error opening source d

In [91]:
dt_m = re.compile(r'\d{8}')
dt_dict = {}

for l in lines:
    dt_key = dt_m.search(l)[0]
    dt_dict[dt_key] = l

In [92]:
sorted(dt_dict.items())

[('20190109',
  '{"log":"[2021-04-09 05:40:10,881] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/1/S2B_30LXK_20190109_0_L2A/B11.tif'),
 ('20190119',
  '{"log":"[2021-04-09 05:40:10,837] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/1/S2B_30LXK_20190119_0_L2A/B8A.tif'),
 ('20190129',
  '{"log":"[2021-04-09 05:40:10,870] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/1/S2B_30LXK_20190129_0_L2A/B8A.tif'),
 ('20190208',
  '{"log":"[2021-04-09 05:40:10,827] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/2/S2B_30LXK_20190208_0_L2A/B8A.tif'),
 ('20190218',
  '{"log":"[2021-04-09 05:40:10,862] {_rio.py:202} ERROR - Error opening source dataset: s3://deafrica-sentinel-2/sentinel-s2-l2a-cogs/30/L/XK/2019/2/S2B_30LXK_20190218_0_L2A/B8A.tif'),
